In [1]:
from dotenv import load_dotenv
load_dotenv()
import textwrap

In [2]:

import time
from PyPDF2 import PdfReader
from PIL import Image

import os


# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
import openai
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import ChatGLM
from langchain.prompts import PromptTemplate


In [49]:
# default endpoint_url for a local deployed ChatGLM api server
endpoint_url = "https://f18b-34-145-4-120.ngrok-free.app"

# direct access endpoint in a proxied environment
# os.environ['NO_PROXY'] = '127.0.0.1'

llm = ChatGLM(
    endpoint_url=endpoint_url,
    max_token=80000,
    history=[],
    top_p=0.9,
)

In [4]:
import io
import requests
from PyPDF2 import PdfReader
headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

url = 'https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf'
response = requests.get(url=url, headers=headers, timeout=120)
on_fly_mem_obj = io.BytesIO(response.content)
pdf_file = PdfReader(on_fly_mem_obj)

In [ ]:
text = ""
for page in pdf_file.pages:
    text += page.extract_text()

In [50]:
# text="""Elon Reeve Musk (/ˈiːlɒn/ EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and chief technology officer of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$222 billion as of December 2023, according to the Bloomberg Billionaires Index, and $244 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.[5][6]

# A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania, and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University. However, Musk dropped out after two days and, with his brother Kimbal, co-founded online city guide software company Zip2. The startup was acquired by Compaq for $307 million in 1999, and, that same year Musk co-founded X.com, a direct bank. X.com merged with Confinity in 2000 to form PayPal.

# In October 2002, eBay acquired PayPal for $1.5 billion, and that same year, with $100 million of the money he made, Musk founded SpaceX, a spaceflight services company. In 2004, he became an early investor in electric vehicle manufacturer Tesla Motors, Inc. (now Tesla, Inc.). He became its chairman and product architect, assuming the position of CEO in 2008. In 2006, Musk helped create SolarCity, a solar-energy company that was acquired by Tesla in 2016 and became Tesla Energy. In 2013, he proposed a hyperloop high-speed vactrain transportation system. In 2015, he co-founded OpenAI, a nonprofit artificial intelligence research company. The following year, Musk co-founded Neuralink—a neurotechnology company developing brain–computer interfaces—and the Boring Company, a tunnel construction company. In 2022, he acquired Twitter for $44 billion. He subsequently merged the company into newly created X Corp. and rebranded the service as X the following year. In March 2023, he founded xAI, an artificial intelligence company.

# Musk has expressed views that have made him a polarizing figure.[7][8][9] He has been criticized for making unscientific and misleading statements, including COVID-19 misinformation, transphobia[10][11][12] and antisemitic conspiracy theories.[7][13][14][15] His ownership of Twitter has been similarly controversial, including; laying off a large number of employees, an increase in hate speech and misinformation and disinformation on the website, as well as changes to Twitter Blue verification. In 2018, the U.S. Securities and Exchange Commission (SEC) sued him for falsely tweeting that he had secured funding for a private takeover of Tesla. To settle the case, Musk stepped down as the chairman of Tesla and paid a $20 million fine.
# """

In [51]:
# # Define the prompt template
# prompt_template =  """
# Based on the given {text}, perform the tasks below:

# Task 1: Generate a common person's name for the given.

# Task 2: Provide a concise 5-bullet point summary. Write down in bullet points.

# Task 3: Evaluate the person's skills and assign a rating on a scale of 1 to 10 for each skill. Use the format:
# - Economics: [rating]/10
# - Mathematics: [rating]/10
# - [Add more skills as needed]

# Return the following information:
# Person Name: [Result of Task 1]
# Summary:
# - [Point 1]
# - [Point 2]
# - [Point 3]
# - [Point 4]
# - [Point 5]
# Skills: [Result of Task 3]
# """


In [52]:
text_splitter = CharacterTextSplitter(
separator="\n",
chunk_size=1000,
chunk_overlap=200,
length_function=len
)

chunks = text_splitter.split_text(text)
    
# create docs
docs = [Document(page_content=t) for t in chunks[:3]]


# show summarize doc
BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="stuff",prompt=BULLET_POINT_PROMPT)
summarized_docs = chain.run(docs)


Created a chunk of size 1033, which is longer than the specified 1000


In [53]:
print("Summary")
print(summarized_docs)


Summary
Person Name: Elon Musk

Summary:

* Businessman and investor
* Founder and CEO of SpaceX and Tesla, Inc.
* Co-founder of Neuralink and OpenAI
* Owner and chairman of X Corp.
* Creator of the Hyperloop (a high-speed vactrain transportation system)
* Proposed the creation of a space station on Mars
* Aimed to accelerate the transition to sustainable energy

Skills:

* Economics: 9/10
* Mathematics: 8/10
* Business: 9/10
* Technology: 10/10
* Innovation: 9/10
* Leadership: 9/10


In [54]:
wrapped_text = textwrap.wrap(summarized_docs, width=100)

# Print the wrapped text
for line in wrapped_text:
    print(line)

Person Name: Elon Musk  Summary:  * Businessman and investor * Founder and CEO of SpaceX and Tesla,
Inc. * Co-founder of Neuralink and OpenAI * Owner and chairman of X Corp. * Creator of the Hyperloop
(a high-speed vactrain transportation system) * Proposed the creation of a space station on Mars *
Aimed to accelerate the transition to sustainable energy  Skills:  * Economics: 9/10 * Mathematics:
8/10 * Business: 9/10 * Technology: 10/10 * Innovation: 9/10 * Leadership: 9/10
